In [32]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import stopwords
from string import punctuation
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cvectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer = TfidfVectorizer(use_idf=True,ngram_range=(1,1))
lem = WordNetLemmatizer()
punct = list(punctuation)
sw = stopwords.words('english')
sp_char = [':','...',"'","''",'``']
# from unidecode import unidecode

In [3]:
df = pd.read_csv('Tweets.csv')

In [4]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
def pre_process(tweet):
    def pos_tag_simplified(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None
    # tweet = unidecode(tweet)
    tweet=tweet.lower()
    tweet = re.sub(r'\d+', '', tweet)
    tokens = word_tokenize(tweet)
    tokens=[i for i in tokens if i.isalpha()]
    cleaned_tokens = [token for token in tokens if token not in sw and token not in punct ]

    #stems=[ps.stem(token) for token in cleaned_tokens]
    tags=nltk.pos_tag(cleaned_tokens)
    wordnet_tagged = list(map(lambda x: (x[0], pos_tag_simplified(x[1])), tags))

    lemmatized=[]
    for word in wordnet_tagged:
        if word[1]!=None:
            lemmatized.append(lem.lemmatize(word[0],word[1]))
        else:
            lemmatized.append(lem.lemmatize(word[0]))
    return lemmatized

In [6]:
df['clean_txt'] = df['text'].apply(pre_process)

In [7]:
df['text'][18]

'I ❤️ flying @VirginAmerica. ☺️👍'

In [8]:
df['clean_txt'][18]

['fly', 'virginamerica']

In [9]:
df['clean_txt'] = df['clean_txt'].apply(lambda x:' '.join(x))

In [11]:
df['clean_txt'][:5]

0                           virginamerica dhepburn say
1    virginamerica plus added commercial experience...
2    virginamerica today must mean need take anothe...
3    virginamerica really aggressive blast obnoxiou...
4                   virginamerica really big bad thing
Name: clean_txt, dtype: object

In [17]:
bow = cvectorizer.fit_transform(df['clean_txt'])

In [30]:
df_bow = pd.DataFrame(bow.toarray(),columns=cvectorizer.get_feature_names_out())

In [31]:
df_bow.to_csv('bow_tweets.csv',index=False)

In [23]:
tfidf = vectorizer.fit_transform(df['clean_txt'])

In [25]:
vectorizer.get_feature_names_out()

array(['aa', 'aaaand', 'aaadvantage', ..., 'zrh', 'zukes', 'zurich'],
      dtype=object)

In [26]:
tfidf = tfidf.todense().tolist()

In [28]:
df_tfidf = pd.DataFrame(tfidf,columns=vectorizer.get_feature_names_out())

In [29]:
df_tfidf.to_csv('tfidf_tweets.csv',index=False)

---

In [88]:
def pre_process(txt_col):
    def pos_tag_simplified(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None
    tkns = [list(word_tokenize(i)) for i in txt_col]
    ltkns = []
    for i in tkns:
        temp = [ii.lower() for ii in i]
        ltkns.append(temp)
    ctkns = []
    for i in ltkns:
        temp = [ii for ii in i if ii not in sw and ii not in punct and ii not in sp_char]
        ctkns.append(temp)
    tgs = []
    for i in ctkns:
        temp = nltk.pos_tag(i)
        tgs.append(temp)
    ctgs = []
    for i in tgs:
        # print(i)
        temp = list(map(lambda x:(x[0],pos_tag_simplified(x[1])),i))
        ctgs.append(temp)
    lemmatized = []
    for i in ctgs:
        temp = [lem.lemmatize(ii[0],ii[1]) if ii[1] != None else lem.lemmatize(ii[0]) for ii in i]
        lemmatized.append(temp)
    # tkns = [i for i in tkns if i not in sw and i not in punct]
    # tgs = nltk.pos_tag(tkns)
    # tgs = list(map(lambda x:(x[0],pos_tag_simplified(x[1])),tagged)
    # lemmatized = [lem.lemmatize(i[0],i[1]) if i[1] != None else lem.lemmatize(i[0]) for i in tgs]
    return lemmatized

In [26]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [41]:
df['pre_processed'] = list('y'*len(df))

In [92]:
cln = pre_process(df['text'])

In [59]:
nltk.pos_tag(cln[1])

[('virginamerica', 'JJ'),
 ('plus', 'CC'),
 ("'ve", 'VBP'),
 ('added', 'VBN'),
 ('commercials', 'NNS'),
 ('experience', 'NN'),
 ('...', ':'),
 ('tacky', 'NN')]

In [94]:
df['pre_processed'] = cln

In [97]:
df['pre_processed']=df['pre_processed'].apply(lambda x:' '.join(x))

In [100]:
df['pre_processed'] = df['pre_processed'].apply(remove_emojis)

In [114]:
df['text'][18]

'I ❤️ flying @VirginAmerica. ☺️👍'

In [115]:
df['pre_processed'][18]

' fly virginamerica '

In [116]:
rem = ["'s","'ve",'""','"',"'","n't"]
ncln = []
for i in cln:
    temp = [ii for ii in i if ii not in rem]
    ncln.append(temp)

In [117]:
df['pre_processed'] = ncln

In [118]:
df['pre_processed']=df['pre_processed'].apply(lambda x:' '.join(x))

In [119]:
df['pre_processed'] = df['pre_processed'].apply(remove_emojis)

In [120]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,pre_processed
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica dhepburn say
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plus add commercial experience t...
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica today must mean need take anothe...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica really aggressive blast obnoxiou...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica really big bad thing


In [93]:
cln

[['virginamerica', 'dhepburn', 'say'],
 ['virginamerica', 'plus', "'ve", 'add', 'commercial', 'experience', 'tacky'],
 ['virginamerica',
  "n't",
  'today',
  'must',
  'mean',
  'need',
  'take',
  'another',
  'trip'],
 ['virginamerica',
  "'s",
  'really',
  'aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guest',
  'face',
  'amp',
  'little',
  'recourse'],
 ['virginamerica', "'s", 'really', 'big', 'bad', 'thing'],
 ['virginamerica',
  'seriously',
  'would',
  'pay',
  '30',
  'flight',
  'seat',
  "n't",
  'play',
  "'s",
  'really',
  'bad',
  'thing',
  'fly',
  'va'],
 ['virginamerica',
  'yes',
  'nearly',
  'every',
  'time',
  'fly',
  'vx',
  '“',
  'ear',
  'worm',
  '”',
  '’',
  'go',
  'away'],
 ['virginamerica',
  'really',
  'miss',
  'prime',
  'opportunity',
  'men',
  'without',
  'hat',
  'parody',
  'https',
  '//t.co/mwpg7grezp'],
 ['virginamerica', 'well', "didn't…but", '-d'],
 ['virginamerica', 'amaze', 'arrive', 'hour', 'early', "'re", 'good'],